In [1]:
import json
import os
from pyquaternion import Quaternion
import pandas as pd
import pickle
import numpy as np
import math
from tqdm import tqdm

from utils.data_dirs import data_dirs
from utils.get_camera_heading import get_camera_heading
from utils.get_camera_position import get_camera_position
from utils.get_camera_rotation import get_camera_rotation
from utils.get_heading_from_north import get_heading_from_north
from utils.get_heading import get_heading
from utils.index import index
from utils.normalize_angle import normalize_angle
from utils.unique import unique

In [2]:
EXPERIMENT = False

In [3]:
base_dir, output_dir, folder, EXPERIMENT_DATA, suffix, *_ = data_dirs(EXPERIMENT)
print(base_dir)
print(output_dir)
print(folder)
print(EXPERIMENT_DATA)
print(suffix)

/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Trainval
/data/apperception-data/processed/nuscenes/full-dataset-v1.0/Trainval
v1.0-trainval
/work/apperception/data/raw/scenic/experiment_data



In [4]:
os.listdir(os.path.join(base_dir, folder))

['ego_pose.json',
 'scene_js.swp',
 'instance.json',
 'map.json',
 'sample_data.json',
 'visibility.json',
 '.scene.json.swx',
 'sample_annotation.json',
 'log.json',
 'sample.json',
 'sensor.json',
 'attribute.json',
 'scene_js.swo',
 'calibrated_sensor.json',
 '.scene.json.swp',
 'scene.json',
 'category.json']

In [5]:
with open(os.path.join(base_dir, folder, 'calibrated_sensor.json')) as f:
    calibrated_sensor_json = json.load(f)

with open(os.path.join(base_dir, folder, 'category.json')) as f:
    category_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample.json')) as f:
    sample_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample_data.json')) as f:
    sample_data_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample_annotation.json')) as f:
    sample_annotation_json = json.load(f)

with open(os.path.join(base_dir, folder, 'instance.json')) as f:
    instance_json = json.load(f)

with open(os.path.join(base_dir, folder, 'scene.json')) as f:
    scene_json = json.load(f)

with open(os.path.join(base_dir, folder, 'ego_pose.json')) as f:
    ego_pose_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sensor.json')) as f:
    sensor_json = json.load(f)

with open(os.path.join(base_dir, folder, 'log.json')) as f:
    log_json = json.load(f)

In [6]:
if EXPERIMENT:
    files = os.listdir(EXPERIMENT_DATA)
    files_set = set(files)
    sample_data_filter = [s for s in sample_data_json if s['filename'].split('/')[2] in files_set]
else:
    sample_data_filter = sample_data_json
sample_data_filter = [s for s in sample_data_filter if s['fileformat'] == 'jpg']

In [7]:
sample_data_filter[0]

{'token': '050322aacb9047f4bc2a734e800c0192',
 'sample_token': '14d5adfe50bb4445bc3aa5fe607691a8',
 'ego_pose_token': '050322aacb9047f4bc2a734e800c0192',
 'calibrated_sensor_token': '2e64b091b3b146a390c2606b9081343c',
 'timestamp': 1531883530762460,
 'fileformat': 'jpg',
 'is_key_frame': False,
 'height': 900,
 'width': 1600,
 'filename': 'sweeps/CAM_FRONT/n015-2018-07-18-11-07-57+0800__CAM_FRONT__1531883530762460.jpg',
 'prev': 'f0c48a4f1d9f435e8d7b18a04d4d621b',
 'next': '3f79b6459436470799616c6abf614398'}

In [8]:
len(log_json)

68

In [9]:
log_json[0]

{'token': '6b6513e6c8384cec88775cae30b78c0e',
 'logfile': 'n015-2018-07-18-11-07-57+0800',
 'vehicle': 'n015',
 'date_captured': '2018-07-18',
 'location': 'singapore-onenorth'}

In [10]:
sample_tokens = unique(sample_data_filter, 'sample_token')
sample_filter = [
    {
        'sample_token': s['token'],
        'scene_token': s['scene_token'],
        'sample_timestamp': s['timestamp'],
    }
    for s in sample_json
    if s['token'] in sample_tokens
]
len(sample_filter)

34149

In [11]:
calibrated_sensor_tokens = unique(sample_data_filter, 'calibrated_sensor_token')
calibrated_sensor_filter = [
    {
        'calibrated_sensor_token': c['token'],
        'camera_translation': c['translation'],
        'camera_rotation': c['rotation'],
        'camera_intrinsic': c['camera_intrinsic'],
        'sensor_token': c['sensor_token']
    }
    for c in calibrated_sensor_json
    if c['token'] in calibrated_sensor_tokens
]
len(calibrated_sensor_filter)

5100

In [12]:
sensor_tokens = unique(calibrated_sensor_filter, 'sensor_token')
sensor_filter = [
    {
        'sensor_token': s['token'],
        'channel': s['channel'],
        'modality': s['modality']
    }
    for s in sensor_json
    if s['token'] in sensor_tokens
]

In [13]:
ego_pose_tokens = unique(sample_data_filter, 'ego_pose_token')
ego_pose_filter = [
    {
        'ego_pose_token': e['token'],
        'ego_translation': e['translation'],
        'ego_rotation': e['rotation'],
    }
    for e in ego_pose_json
    if e['token'] in ego_pose_tokens
]
len(ego_pose_filter)

1183790

In [14]:
scene_tokens = unique(sample_filter, 'scene_token')
scene_filter = [
    {
        'scene_token': s['token'],
        'scene_name': s['name'],
        'log_token': s['log_token'],
    }
    for s in scene_json
    if s['token'] in scene_tokens
]
len(scene_filter)

850

In [15]:
log_tokens = unique(scene_filter, 'log_token')
log_filter = [
    {
        'log_token': l['token'],
        'location': l['location'],
    }
    for l in log_json
    if l['token'] in log_tokens
]
len(log_filter)

68

In [16]:
log_map = index(log_filter, 'log_token')
sample_map = index(sample_filter, 'sample_token')
calibrated_sensor_map = index(calibrated_sensor_filter, 'calibrated_sensor_token')
ego_pose_map = index(ego_pose_filter, 'ego_pose_token')
scene_map = index(scene_filter, 'scene_token')
sensor_map = index(sensor_filter, 'sensor_token')

In [17]:
def s_map(s):
    sample = sample_map[s['sample_token']]
    calibrated_sensor = calibrated_sensor_map[s['calibrated_sensor_token']]
    ego_pose = ego_pose_map[s['ego_pose_token']]
    scene = scene_map[sample['scene_token']]
    sensor = sensor_map[calibrated_sensor['sensor_token']]
    assert sensor['modality'] == 'camera'
    
    log = log_map[scene['log_token']]
    
    ego_heading = get_heading_from_north(Quaternion(ego_pose['ego_rotation']))
    camera_heading = get_camera_heading(Quaternion(calibrated_sensor['camera_rotation']))
    ret = {
        **s,
        **sample,
        **calibrated_sensor,
        **ego_pose,
        **scene,
        **sensor,
        **log,
        'ego_heading': ego_heading * 180 / math.pi,
        'camera_heading': normalize_angle(camera_heading + ego_heading) * 180 / math.pi,
        'camera_translation': get_camera_position(
            calibrated_sensor['camera_translation'],
            ego_pose['ego_translation'],
            ego_pose['ego_rotation'],
        ),
        'camera_rotation': get_camera_rotation(
            calibrated_sensor['camera_rotation'],
            ego_pose['ego_rotation'],
        ),
    }
    del ret['ego_pose_token']
    del ret['calibrated_sensor_token']
    del ret['log_token']
    del ret['fileformat']
    del ret['height']
    del ret['width']
    del ret['prev']
    del ret['next']
    del ret['scene_token']
    del ret['sensor_token']
    del ret['modality']
    return ret

sample_data_res = [*tqdm(map(s_map, sample_data_filter), total=len(sample_data_filter))]

len(sample_data_res)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1183790/1183790 [04:00<00:00, 4923.72it/s]


1183790

In [18]:
sample_data_res[0]

{'token': '050322aacb9047f4bc2a734e800c0192',
 'sample_token': '14d5adfe50bb4445bc3aa5fe607691a8',
 'timestamp': 1531883530762460,
 'is_key_frame': False,
 'filename': 'sweeps/CAM_FRONT/n015-2018-07-18-11-07-57+0800__CAM_FRONT__1531883530762460.jpg',
 'sample_timestamp': 1531883530949817,
 'camera_translation': array([1010.28536588,  613.79584851,    1.55080497]),
 'camera_rotation': array([-0.71340144,  0.7006582 , -0.00763616,  0.00884084]),
 'camera_intrinsic': [[1266.417203046554, 0.0, 816.2670197447984],
  [0.0, 1266.417203046554, 491.50706579294757],
  [0.0, 0.0, 1.0]],
 'ego_translation': [1010.2539305089451, 612.1315307276016, 0.0],
 'ego_rotation': [-0.7172259417302351,
  -0.008052965142693864,
  0.008687339134831436,
  -0.6967400005743498],
 'scene_name': 'scene-0001',
 'channel': 'CAM_FRONT',
 'location': 'singapore-onenorth',
 'ego_heading': -1.6592785295958965,
 'camera_heading': -1.3335577747511875}

In [19]:
for sa in sample_annotation_json:
    if sa['sample_token'] not in sample_tokens:
        print(sa['sample_token'])

In [20]:
sample_annotation_filter = [
    sa
    for sa in sample_annotation_json
    if sa['sample_token'] in sample_tokens
]
len(sample_annotation_filter)

1166187

In [21]:
instance_tokens = unique(sample_annotation_filter, 'instance_token')
instance_filter = [
    {
        'instance_token': i['token'],
        'category_token': i['category_token']
    }
    for i in instance_json
    if i['token'] in instance_tokens
]
len(instance_filter)

64386

In [22]:
category_tokens = unique(instance_filter, 'category_token')
category_filter = [
    {
        'category_token': c['token'],
        'category': c['name']
    }
    for c in category_json
    if c['token'] in category_tokens
]
len(category_filter)

23

In [23]:
instance_map = index(instance_filter, 'instance_token')
category_map = index(category_filter, 'category_token')

In [24]:
def sa_map(sa):
    instance = instance_map[sa['instance_token']]
    sample = sample_map[sa['sample_token']]
    scene = scene_map[sample['scene_token']]
    log = log_map[scene['log_token']]
    ret = {
        **sa,
        **instance,
        **category_map[instance['category_token']],
        'heading': (get_heading_from_north(Quaternion(sa['rotation']))) * 180 / math.pi,
        'location': log['location'],
        'scene_name': scene['scene_name']
    }
    
    del ret['visibility_token']
    del ret['attribute_tokens']
    del ret['prev']
    del ret['next']
    del ret['num_lidar_pts']
    del ret['num_radar_pts']
    del ret['category_token']
    
    return ret

sample_annotation_res = [*tqdm(map(sa_map, sample_annotation_filter), total=len(sample_annotation_filter))]
len(sample_annotation_res)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166187/1166187 [00:26<00:00, 43780.14it/s]


1166187

In [25]:
df_sample_data = pd.DataFrame(sample_data_res)
print(len(df_sample_data))
df_sample_data[:1]

1183790


,token,sample_token,timestamp,is_key_frame,filename,sample_timestamp,camera_translation,camera_rotation,camera_intrinsic,ego_translation,ego_rotation,scene_name,channel,location,ego_heading,camera_heading
0,050322aacb9047f4bc2a734e800c0192,14d5adfe50bb4445bc3aa5fe607691a8,1531883530762460,False,sweeps/CAM_FRONT/n015-2018-07-18-11-07-57+0800...,1531883530949817,"[1010.2853658775477, 613.795848507104, 1.55080...","[-0.7134014433076645, 0.7006581972076307, -0.0...","[[1266.417203046554, 0.0, 816.2670197447984], ...","[1010.2539305089451, 612.1315307276016, 0.0]","[-0.7172259417302351, -0.008052965142693864, 0...",scene-0001,CAM_FRONT,singapore-onenorth,-1.659279,-1.333558


In [26]:
df_sample_annotation = pd.DataFrame(sample_annotation_res)
print(len(df_sample_annotation))
df_sample_annotation[:1]

1166187


,token,sample_token,instance_token,translation,size,rotation,category,heading,location,scene_name
0,2cd832644d09479389ed0785e5de85c9,c36eb85918a84a788e236f5c9eef2b05,5e2b6fd1fab74d04a79eefebbec357bb,"[993.884, 612.441, 0.675]","[0.3, 0.291, 0.734]","[-0.04208490861058176, 0.0, 0.0, 0.99911403776...",movable_object.trafficcone,94.824,singapore-onenorth,scene-0001


In [27]:
df_sample_data_keyframe = (df_sample_data
    [df_sample_data["is_key_frame"]]
    [["token", "sample_token"]]
    .groupby('sample_token')
    .agg(list)
    .reset_index()
    .rename(columns={'token': 'sample_data_tokens'})
)

df_sample_annotation = (df_sample_annotation
    .set_index("sample_token")
    .join(
        df_sample_data_keyframe.set_index("sample_token"),
        on="sample_token",
    )
    .reset_index()
)
print(len(df_sample_annotation))
df_sample_annotation[:1]

1166187


,sample_token,token,instance_token,translation,size,rotation,category,heading,location,scene_name,sample_data_tokens
0,c36eb85918a84a788e236f5c9eef2b05,2cd832644d09479389ed0785e5de85c9,5e2b6fd1fab74d04a79eefebbec357bb,"[993.884, 612.441, 0.675]","[0.3, 0.291, 0.734]","[-0.04208490861058176, 0.0, 0.0, 0.99911403776...",movable_object.trafficcone,94.824,singapore-onenorth,scene-0001,"[7ad5d6b946ec4c8daf9ce2938e419ba7, e2310bef3df..."


In [28]:
len(df_sample_annotation)

1166187

In [29]:
len(df_sample_data_keyframe["sample_data_tokens"].tolist())

34149

In [30]:
len({*df_sample_data_keyframe["sample_token"].tolist()})
len(df_sample_data_keyframe["sample_token"].tolist())

34149

In [31]:
df_sample_data["frame_order"] = (df_sample_data
    .groupby('scene_name')['timestamp']
    .rank(method='first')
    .astype(int)
)

In [32]:
print(len(df_sample_data))

1183790


In [33]:
print(len(df_sample_annotation))

1166187


In [34]:
df_sample_data.to_pickle(os.path.join(output_dir, f'sample_data{suffix}.pkl'))
df_sample_annotation.to_pickle(os.path.join(output_dir, f'annotation{suffix}.pkl'))

In [35]:
# sample_data_final.to_csv(os.path.join(output_dir, f"sample_data{suffix}.csv"), index=False)

In [36]:
# sample_annotation_final.to_csv(os.path.join(output_dir, f"annotation{suffix}.csv"), index=False)

# Experiment with the data + some validation

In [37]:
(df_sample_data[df_sample_data['is_key_frame']]['timestamp'] - df_sample_data[df_sample_data['is_key_frame']]['sample_timestamp']) / 1_000_000

1         -0.037737
3         -0.036907
9         -0.037503
13        -0.037357
16        -0.036216
             ...   
1183767   -0.043393
1183775   -0.043458
1183777   -0.043966
1183786   -0.042987
1183789   -0.043593
Length: 204894, dtype: float64

In [38]:
df_sample_data[df_sample_data['is_key_frame']].groupby('sample_token').agg(
    timestamp_len=pd.NamedAgg(column="timestamp", aggfunc=lambda x: len(set(x))),
    timestamp_avg=pd.NamedAgg(column="timestamp", aggfunc=lambda x: f"{sum(x) / len(x):.5f}"),
    timestamp_range=pd.NamedAgg(column="timestamp", aggfunc=lambda x: max(x) - min(x)),
    timestamp_max=pd.NamedAgg(column="timestamp", aggfunc=lambda x: max(x)),
    timestamp_min=pd.NamedAgg(column="timestamp", aggfunc=lambda x: min(x)),
    sample_timestamp=pd.NamedAgg(column="sample_timestamp", aggfunc=lambda x: set(x))
)


# .agg({
#     'timestamp': (lambda x: len(set(x))),
#     'timestamp': (lambda x: sum(x) / len(x))
# })

,timestamp_len,timestamp_avg,timestamp_range,timestamp_max,timestamp_min,sample_timestamp
sample_token,,,,,,
00014584c6de4789a69b4717fa3c0d22,6,1538033994275080.75000,42579,1538033994297423,1538033994254844,{1538033994297142}
0003b4042e73443d81e17dc6f5b14de3,6,1537290912875126.75000,42606,1537290912897405,1537290912854799,{1537290912898341}
000681a060c04755a1537cf83b53ba57,6,1531281560175081.75000,42579,1531281560197423,1531281560154844,{1531281560199908}
000868a72138448191b4092f75ed7776,6,1533151882525126.75000,42606,1533151882547405,1533151882504799,{1533151882547619}
0008b7653d1a42179ac9c222e734bcef,6,1537299517425126.75000,42606,1537299517447405,1537299517404799,{1537299517449730}
...,...,...,...,...,...,...
fff7106ca1f34c7a83e72ffbc0c0b1b5,6,1537296206625126.75000,42606,1537296206647405,1537296206604799,{1537296206649326}
fff7244095b441d6a053da2951cf2b3b,6,1542801265425080.75000,42579,1542801265447423,1542801265404844,{1542801265448126}
fff7416f03794572b82bc42d4fc7093d,6,1532663148175080.75000,42579,1532663148197423,1532663148154844,{1532663148198418}


In [39]:
42606 / 1_000_000

0.042606

In [40]:
df_sample_data[df_sample_data['timestamp'] == df_sample_data['sample_timestamp']]

,token,sample_token,timestamp,is_key_frame,filename,sample_timestamp,camera_translation,camera_rotation,camera_intrinsic,ego_translation,ego_rotation,scene_name,channel,location,ego_heading,camera_heading,frame_order
88197,d3aff8de619b495b8f7f7bedd9ac92f6,1a2fa42ebe374ca4bdb234d8b9bc0ad7,1533153439147405,True,samples/CAM_BACK_LEFT/n008-2018-08-01-15-52-19...,1533153439147405,"[1949.0928978847091, 870.9364965073557, 1.5523...","[-0.6939571881653821, 0.6996558611738033, 0.11...","[[1254.9860565800168, 0.0, 829.5769333630991],...","[1948.0600093886542, 870.3921681493506, 0.0]","[-0.999863886828878, 0.013832570595238017, 0.0...",scene-0067,CAM_BACK_LEFT,boston-seaport,-89.002501,19.570889,625
434744,d3c8623c857b4f79b328070c1e99de10,5cca256ed9ac40a1ac2f91fe485a4d5c,1538033647297423,True,samples/CAM_BACK_LEFT/n015-2018-09-27-15-33-17...,1538033647297423,"[599.7696559975788, 1995.8120655161458, 1.5997...","[0.16967847715037687, -0.16587495589955784, -0...","[[1256.7414812095406, 0.0, 792.1125740759628],...","[600.8363571699199, 1995.4354757460915, 0.0]","[0.39112586315815345, 0.0043199748487511, -0.0...",scene-0400,CAM_BACK_LEFT,singapore-hollandvillage,43.947899,152.542952,402
520920,497993e6c5564c7a9810dfc7aa290899,0a566bf5174241078ac082913e9763ea,1537289882547419,True,samples/CAM_BACK_LEFT/n008-2018-09-18-12-53-31...,1537289882547419,"[1315.8283544488515, 1038.2258572601168, 1.562...","[-0.13355111389173654, 0.13336922972646031, 0....","[[1254.9860565800168, 0.0, 829.5769333630991],...","[1316.9507949017625, 1037.9601252189689, 0.0]","[-0.3447495981725934, -0.0017739041791563614, ...",scene-0464,CAM_BACK_LEFT,boston-seaport,49.667954,158.241343,77
666814,cda735ed257e4a839dbb076c60d77995,924154c08a8549c3b1e4a7e20aa0ab77,1537295814897405,True,samples/CAM_BACK_LEFT/n008-2018-09-18-14-35-12...,1537295814897405,"[1565.1119874339486, 1328.1061335613247, 1.584...","[0.7177714448087549, -0.6950076194590807, 0.02...","[[1254.9860565800168, 0.0, 829.5769333630991],...","[1563.9884634395687, 1328.0941376888122, 0.0]","[0.9793259846878615, 0.007183603355500013, -0....",scene-0625,CAM_BACK_LEFT,boston-seaport,-113.301615,-4.728226,77
850770,9d1e2a81c71d43288e1f2be78c32ec6d,6e54dfd8cdf34ef7bb61e2284e9911b5,1538449267447423,True,samples/CAM_BACK_LEFT/n015-2018-10-02-10-56-37...,1538449267447423,"[1692.23825799362, 2723.1729223912494, 1.56304...","[-0.6614956534299359, 0.6853718191713943, -0.2...","[[1256.7414812095406, 0.0, 792.1125740759628],...","[1691.1877622048573, 2723.7134991905714, 0.0]","[-0.8907746669439669, 0.010802745393030054, -0...",scene-0791,CAM_BACK_LEFT,singapore-queenstown,-144.021243,-35.426189,650
898462,c4695bac29764181881a7aedfe857f35,02c953a65a614ba6a0b64d78111bc8f4,1538985963147423,True,samples/CAM_BACK_LEFT/n015-2018-10-08-16-03-24...,1538985963147423,"[1835.8346365833024, 1510.4861359491176, 1.603...","[0.3600013392295245, -0.3769074554876809, 0.60...","[[1256.7414812095406, 0.0, 792.1125740759628],...","[1836.2235304067606, 1511.5429558162184, 0.0]","[0.3763837187490786, -0.00594234457585575, -0....",scene-0854,CAM_BACK_LEFT,singapore-queenstown,134.222845,-117.182102,953
1068329,107e469c2f0743858ff1009505d9bb7e,04cebd7db4ce44369523b4f3f2ec6ce4,1542193584447423,True,samples/CAM_BACK_LEFT/n015-2018-11-14-18-57-54...,1542193584447423,"[1053.1738261449236, 1728.7353151441898, 1.591...","[0.6937276111803539, -0.7183465147075474, -0.0...","[[1256.7414812095406, 0.0, 792.1125740759628],...","[1052.0648287529004, 1728.4589602723213, 0.0]","[0.9938772719007054, -0.009076941081439744, -0...",scene-1009,CAM_BACK_LEFT,singapore-queenstown,-102.641060,5.953993,214
1132870,b67b067d9b59458b94cea7b08135cf16,df9df601e42b4186a01182d1841e2a09,1542799756947423,True,samples/CAM_BACK_LEFT/n015-2018-11-21-19-21-35...,1542799756947423,"[616.7363470933337, 2027.1991738556992, 1.6323...","[0.40327675398772317, -0.4760601946017423, 0.5...","[[1256.7414812095406, 0.0, 792.1125740759628],...","[616.7213875077988, 2028.2827698918913, 0.0]","[0.487

In [41]:
df = df_sample_data.copy(deep=True)
df['timediff'] = df['sample_timestamp'] - df['timestamp']
df['timediffabs'] = abs(df['sample_timestamp'] - df['timestamp'])

In [42]:
df[(df['timediff'] < 0) & (df['is_key_frame'] != True)]

,token,sample_token,timestamp,is_key_frame,filename,sample_timestamp,camera_translation,camera_rotation,camera_intrinsic,ego_translation,ego_rotation,scene_name,channel,location,ego_heading,camera_heading,frame_order,timediff,timediffabs


In [43]:
df1 = df.sort_values(['is_key_frame']).groupby(['sample_token', 'channel']).agg(
#     timestamp_len=pd.NamedAgg(column="timestamp", aggfunc=lambda x: len(set(x))),
    timestamp_arr=pd.NamedAgg(column="timediff", aggfunc=lambda x: list(x)),
    iskeyframe_arr=pd.NamedAgg(column="is_key_frame", aggfunc=lambda x: list(x)),
    keyframe_closest=pd.NamedAgg(column="timediff", aggfunc=lambda x: all(xx > x.iloc[-1] for xx in x)),
)

df1

timestamp_arr  \
sample_token                     channel                                                            
00014584c6de4789a69b4717fa3c0d22 CAM_BACK          [159617, 409617, 359617, 109617, 259617, 9617]   
                                 CAM_BACK_LEFT      [249719, 349719, 99719, 399719, 149719, -281]   
                                 CAM_BACK_RIGHT   [269249, 369249, 419249, 169249, 119249, 19249]   
                                 CAM_FRONT        [434682, 384682, 284682, 184682, 134682, 34681]   
                                 CAM_FRONT_LEFT   [142298, 392298, 442298, 292298, 192298, 42298]   
...                                                                                           ...   
fffce4445c964803a12a2d64023fde40 CAM_BACK_LEFT      [49560, 299560, 399560, 149560, 249560, -440]   
                                 CAM_BACK_RIGHT    [319090, 419090, 69090, 169090, 269090, 19090]   
                                 CAM_FRONT         [334523, 184523, 84523, 284523, 434523, 34523]   
                                 CAM_FRONT_LEFT    [442139, 92139, 342139, 292139, 192139, 42139]   
                                 CAM_FRONT_RIGHT   [276644, 326644, 426063, 76644, 176644, 26644]   

                                                                             iskeyframe_arr  \
sample_token                     channel                                                      
00014584c6de4789a69b4717fa3c0d22 CAM_BACK         [False, False, False, False, False, True]   
                                 CAM_BACK_LEFT    [False, False, False, False, False, True]   
                                 CAM_BACK_RIGHT   [False, False, False, False, False, True]   
                                 CAM_FRONT        [False, False, False, False, False, True]   
                                 CAM_FRONT_LEFT   [False, False, False, False, False, True]   
...                                                                                     ...   
fffce4445c964803a12a2d64023fde40 CAM_BACK_LEFT    [False, False, False, False, False, True]   
                                 CAM_BACK_RIGHT   [False, False, False, False, False, True]   
                                 CAM_FRONT        [False, False, False, False, False, True]   
                                 CAM_FRONT_LEFT   [False, False, False, False, False, True]   
                                 CAM_FRONT_RIGHT  [False, False, False, False, False, True]   

                                                  keyframe_closest  
sample_token                     channel                            
00014584c6de4789a69b4717fa3c0d22 CAM_BACK                    False  
                                 CAM_BACK_LEFT               False  
                                 CAM_BACK_RIGHT              False  
                                 CAM_FRONT                   False  
                                 CAM_FRONT_LEFT              False  
...                                                            ...  
fffce4445c964803a12a2d64023fde40 CAM_BACK_LEFT               False  
                                 CAM_BACK_RIGHT              False  
                                 CAM_FRONT                   False  
                                 CAM_FRONT_LEFT              False  
                                 CAM_FRONT_RIGHT             False  

[204894 rows x 3 columns]